<a href="https://colab.research.google.com/github/mrcroney/amini-technical-assignment/blob/main/health_registry_eda_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
message = "Hello Amini World!"
print(message)

Hello Amini World!
